##Label


In [1]:
import numpy as np

In [16]:
class UnionFind:
    def __init__(self, N):
        self.parent = np.array([-1] * (2*N-1))
        self.next_label = N
        
    def union(self, m, n):
        self.parent[m] = self.next_label
        self.parent[n] = self.next_label
        ++self.next_label
        
    def find(self, n):
        while self.parent[n] != -1 :
            n = self.parent[n]
        return n
    
    def fast_find(self, n):
        p = n
        while self.parent[n] != -1 :
            n = self.parent[n]
        while p != n: # label all the way up to the root
            (p, self.parent[p]) = (self.parent[p], n)

In [18]:
def label(L):
    LL = []
    N = len(L) + 1
    U = UnionFind(N)
    for (a, b, delta) in L:
        LL.append((U.fast_find(a), U.fast_find(b), delta))
        U.union(a, b)
    return np.array(LL)